# Embryo Prescription Query

This notebook queries embryo data and associated prescriptions for a given prontuario.

## Tables Queried:
- **Huntington Data Lake** (huntington_data_lake.duckdb):
  - `gold.planilha_embryoscope_combined` (source embryos)
  - `gold.embryos_with_prescription_long` (final joined result)
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - `silver.view_medicamentos_prescricoes` (source prescriptions)


In [67]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

## Database Connections

In [68]:
def get_connections():
    """Get connections to required databases"""
    repo_root = os.path.join('..', '..')
    lake_path = os.path.join(repo_root, 'database', 'huntington_data_lake.duckdb')
    clinisys_path = os.path.join(repo_root, 'database', 'clinisys_all.duckdb')
    
    lake_con = duckdb.connect(lake_path, read_only=True)
    clinisys_con = duckdb.connect(clinisys_path, read_only=True)
    
    return lake_con, clinisys_con

## Query Functions

In [69]:
def query_embryoscope_combined(conn, prontuario):
    """Query gold.planilha_embryoscope_combined for a prontuario"""
    print(f"\n=== GOLD.PLANILHA_EMBRYOSCOPE_COMBINED ===")
    query = f"""
    SELECT *
    FROM gold.planilha_embryoscope_combined 
    WHERE micro_prontuario = {prontuario}
    ORDER BY oocito_id DESC
    """
    try:
        df = conn.execute(query).df()
        print(f"Found {len(df)} embryos.")
        return df
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

def query_clinisys_prescriptions(conn, prontuario):
    """Query silver.view_medicamentos_prescricoes for a prontuario"""
    print(f"\n=== SILVER.VIEW_MEDICAMENTOS_PRESCRICOES ===")
    query = f"""
    SELECT *
    FROM silver.view_medicamentos_prescricoes 
    WHERE prontuario = {prontuario}
    ORDER BY ficha_id DESC, data_inicial ASC
    """
    try:
        df = conn.execute(query).df()
        print(f"Found {len(df)} prescriptions.")
        return df
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

def query_final_long(conn, prontuario):
    """Query the final generated gold table"""
    print(f"\n=== GOLD.EMBRYOS_WITH_PRESCRIPTION_LONG ===")
    query = f"""
    SELECT *
    FROM gold.embryos_with_prescription_long
    WHERE micro_prontuario = {prontuario}
    ORDER BY oocito_id DESC, presc_data_inicial ASC
    """
    try:
        df = conn.execute(query).df()
        print(f"Found {len(df)} join results (long format).")
        return df
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

def query_final_wide(conn, prontuario):
    """Query the final generated gold table"""
    print(f"\n=== GOLD.EMBRYOS_WITH_PRESCRIPTION_WIDE ===")
    query = f"""
    SELECT *
    FROM gold.embryos_with_prescription_wide
    WHERE micro_prontuario = {prontuario}
    ORDER BY oocito_id DESC
    """
    try:
        df = conn.execute(query).df()
        print(f"Found {len(df)} join results (long format).")
        return df
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

## Execution

In [70]:
# SET THE PRONTUARIO HERE
prontuario_to_query = 828439

lake_con, clinisys_con = get_connections()

df_embryos = query_embryoscope_combined(lake_con, prontuario_to_query)
df_presc = query_clinisys_prescriptions(clinisys_con, prontuario_to_query)
df_long = query_final_long(lake_con, prontuario_to_query)
df_wide = query_final_wide(lake_con, prontuario_to_query)

lake_con.close()
clinisys_con.close()


=== GOLD.PLANILHA_EMBRYOSCOPE_COMBINED ===
Found 27 embryos.

=== SILVER.VIEW_MEDICAMENTOS_PRESCRICOES ===
Found 17 prescriptions.

=== GOLD.EMBRYOS_WITH_PRESCRIPTION_LONG ===
Found 6 join results (long format).

=== GOLD.EMBRYOS_WITH_PRESCRIPTION_WIDE ===
Found 2 join results (long format).


## Display Results

In [71]:
df_embryos

,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,oocito_ResultadoPGD,oocito_ResultadoPGDDetalhes,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od,nome_medico,tipo_medico,flag_is_external_medical,prontuario,patient_PatientID,patient_PatientIDx,patient_FirstName,patient_LastName,patient_DateOfBirth,patient_YearOfBirth,patient_unit_huntington,treatment_TreatmentName,embryo_EmbryoID,embryo_EmbryoDate,embryo_embryo_number,embryo_Position,embryo_InstrumentNumber,embryo_WellNumber,embryo_FertilizationTime,embryo_FertilizationMethod,embryo_EmbryoFate,embryo_Description,embryo_EmbryoDescriptionID,embryo_KIDScore,embryo_KIDDate,embryo_KIDVersion,embryo_KIDUser,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Value_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Value_EVEN2,embryo_Timestamp_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Value_EVEN4,embryo_Timestamp_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Value_EVEN8,embryo_Timestamp_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Value_FRAG2,embryo_Timestamp_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Value_FRAG4,embryo_Timestamp_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Value_FRAG8,embryo_Timestamp_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Value_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Value_ICM,embryo_Timestamp_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Value_Line,embryo_Timestamp_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Value_MN2Type,embryo_Timestamp_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Value_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Value_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Value_PN,embryo_Timestamp_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Value_Pulsing,embryo_Timestamp_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Value_Strings,embryo_Timestamp_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Value_TE,embryo_Timestamp_TE,embryo_Name_t2,embryo_Time_t2,embryo_Value_t2,embryo_Timestamp_t2,embryo_Name_t3,embryo_Time_t3,embryo_Value_t3,embryo_Timestamp_t3,embryo_Name_t4,embryo_Time_t4,embryo_Value_t4,embryo_Timestamp_t4,embryo_Name_t5,embryo_Time_t5,embryo_Value_t5,embryo_Timestamp_t5,embryo_Name_t6,embryo_Time_t6,embryo_Value_t6,embryo_Timestamp_t6,embryo_Name_t7,embryo_Time_t7,embryo_Value_t7,embryo_Timestamp_t7,embryo_Name_t8,embryo_Time_t8,embryo_Value_t8,embryo_Timestamp_t8,embryo_Name_t9,embryo_Time_t9,embryo_Value_t9,embryo_Timestamp_t9,embryo_Name_tB,embryo_Time_tB,embryo_Value_tB,embryo_Timestamp_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Value_tEB,embryo_Timestamp_tEB,embryo_Name_tHB,embry

In [72]:
df_presc

,id,prontuario,ficha_tipo,ficha_id,data,via,hora,intervalo,observacoes,data_inicial,data_final,observacao_medicamento,quantidade,forma,duracao,hash,extraction_timestamp,medicamento,unidade,med_nome,dose,unidade_padronizada,numero_dias,grupo_medicamento,dose_total
0,921829,828439,Tratamento,40582,2026-01-06,Via oral,1900-01-01 10:00:00,12.0,Administrar conforme tabela abaixo,2026-01-06,2026-02-05,None,0,None,0,ca52b88de32bee1bcaea59391c2c4536,2026-02-05 19:18:47,30,mg,PRIMOGYNA - 2 mg cprs. rev. est.cal. x 28,4.0,mg,31,PRIMOGYNA,248.0
1,897334,828439,Tratamento,40582,2026-01-06,Via oral,1900-01-01 10:00:00,24.0,Tomar uma vez ao dia,2026-01-06,2026-01-19,None,0,None,0,f4bb1b0ee615473acd8c16a6fd1f47db,2026-01-16 19:35:06,48,mg,AAS - 100 mg,100.0,mg,14,AAS,1400.0
2,897333,828439,Tratamento,40582,2026-01-06,Via oral,1900-01-01 10:00:00,12.0,Administrar conforme tabela abaixo,2026-01-06,2026-01-19,None,0,None,0,4e2be51dd3e21a16553fce2d49a8036c,2026-01-16 19:35:06,30,mg,PRIMOGYNA - 2 mg cprs. rev. est.cal. x 28,4.0,mg,14,PRIMOGYNA,112.0
3,921830,828439,Tratamento,40582,2026-01-06,Via oral,1900-01-01 10:00:00,24.0,Tomar uma vez ao dia,2026-01-06,2026-02-05,None,0,None,0,12683e63116c8bb41bab730def0825d9,2026-02-05 19:18:47,48,mg,AAS - 100 mg,100.0,mg,31,AAS,3100.0
4,897335,828439,Tratamento,40582,2026-01-06,Via vaginal,1900-01-01 22:00:00,24.0,10 óvulos vaginais com: Lactobacillus gasseri 1 x 109 ufc/unidade Lactobacillus rhamnosus 1 x 109 ufc/unidade Lactobacillus Crispatus 1 x 109 ufc/unidade Lactobacillus Plantarum 1 x 109 ufc/unidade. Aplicar um óvulo vaginal durante 10 noites,2026-01-08,2026-01-17,None,0,None,0,1e339948d25ba3ca34df8859f1fdfa8f,2026-01-16 19:35:06,41,Ovulo Vaginal,LACTOBACILLUS,1.0,ovulo,10,LACTOBACILLUS,10.0
5,921831,828439,Tratamento,40582,2026-01-06,Via vaginal,1900-01-01 22:00:00,24.0,10 óvulos vaginais com: Lactobacillus gasseri 1 x 109 ufc/unidade Lactobacillus rhamnosus 1 x 109 ufc/unidade Lactobacillus Crispatus 1 x 109 ufc/unidade Lactobacillus Plantarum 1 x 109 ufc/unidade. Aplicar um óvulo vaginal durante 10 noites,2026-01-08,2026-01-17,None,0,None,0,c109690711dc29a004ac62ec3e026305,2026-02-05 19:18:47,41,Ovulo Vaginal,LACTOBACILLUS,1.0,ovulo,10,LACTOBACILLUS,10.0
6,897336,828439,Tratamento,40582,2026-01-14,Via transdérmica,1900-01-01 10:00:00,12.0,Passar na face interna do antebraço conforme tabela abaixo,2026-01-14,2026-01-19,None,0,None,0,b2c375e1b4dd2163ca4697866ddea6eb,2026-01-16 19:35:06,33,pump,OESTROGEL- Fr (dosador) c/80g gel,2.0,pump,6,OESTROGEL,24.0
7,921832,828439,Tratamento,40582,2026-01-14,Via transdérmica,1900-01-01 10:00:00,12.0,Passar na face interna do antebraço conforme tabela abaixo,2026-01-14,2026-02-05,None,0,None,0,b70da249c12db81d60fb8135ac94fce5,2026-02-05 19:18:47,33,pump,OESTROGEL- Fr (dosador) c/80g gel,2.0,pump,23,OESTROGEL,92.0
8,921833,828439,Tratamento,40582,2026-01-20,Via vaginal,1900-01-01 10:00:00,12.0,Administrar conforme tabela abaixo.,2026-01-21,2026-01-23,None,0,None,0,57d1f01a1c7a67f80056d3d822af6373,2026-02-05 19:18:47,25,mg,UTROGESTAN - 200 mg,400.0,mg,3,UTROGESTAN,2400.0
9,921834,828439,Tratamento,40582,2026-01-20,Via vaginal,1900-01-01 22:00:00,24.0,Administrar 1 aplicador via vaginal ao deitar. Manter até o dia do teste de gravidez. Em caso positivo continuar o uso.,2026-01-21,2026-02-05,None,0,None,0,f143403f3fde1fd9217b8233501421ff,2026-02-05 19:18:47,29,mg,CRINONE 8% - Ct c/15 aplicadores de 90 mg,90.0,mg,16,CRINONE,1440.0


In [73]:
df_presc[df_presc['ficha_id']==28105]

,id,prontuario,ficha_tipo,ficha_id,data,via,hora,intervalo,observacoes,data_inicial,data_final,observacao_medicamento,quantidade,forma,duracao,hash,extraction_timestamp,medicamento,unidade,med_nome,dose,unidade_padronizada,numero_dias,grupo_medicamento,dose_total
14,511425,828439,Tratamento,28105,2024-07-12,Via subcutânea,1900-01-01 22:30:00,24.0,Administrar conforme tabela abaixo.,2024-07-13,2024-07-22,None,0,None,0,7cfe27bf3a6f40fc006d6123a4ce1923,2025-08-20 19:58:32,12,UI,GONAL,150.0,UI,10,GONAL,1500.0
15,511426,828439,Tratamento,28105,2024-07-12,Via oral,1900-01-01 10:30:00,12.0,Administrar conforme tabela abaixo,2024-07-13,2024-07-23,None,0,None,0,e88fab79dffc6b1d5a30b558cbb174cb,2025-08-20 19:58:32,27,mg,DUPHASTON - 10 MG,10.0,mg,11,DUPHASTON,220.0
16,511427,828439,Tratamento,28105,2024-07-22,Via subcutânea,1900-01-01 00:40:00,24.0,Na madrugada de terça para quarta (23 - 24 de julho),2024-07-24,2024-07-24,None,0,None,0,916b286435cc062c3dc14505e39e9e86,2025-08-20 19:58:32,22,ampola,"GONAPEPTYL DAILY - ampola 0,1 mg/ml",3.0,ampola,1,GONAPEPTYL,3.0


In [74]:
df_long

,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,oocito_ResultadoPGD,oocito_ResultadoPGDDetalhes,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od,nome_medico,tipo_medico,flag_is_external_medical,prontuario,patient_PatientID,patient_PatientIDx,patient_FirstName,patient_LastName,patient_DateOfBirth,patient_YearOfBirth,patient_unit_huntington,treatment_TreatmentName,embryo_EmbryoID,embryo_EmbryoDate,embryo_embryo_number,embryo_Position,embryo_InstrumentNumber,embryo_WellNumber,embryo_FertilizationTime,embryo_FertilizationMethod,embryo_EmbryoFate,embryo_Description,embryo_EmbryoDescriptionID,embryo_KIDScore,embryo_KIDDate,embryo_KIDVersion,embryo_KIDUser,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Value_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Value_EVEN2,embryo_Timestamp_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Value_EVEN4,embryo_Timestamp_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Value_EVEN8,embryo_Timestamp_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Value_FRAG2,embryo_Timestamp_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Value_FRAG4,embryo_Timestamp_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Value_FRAG8,embryo_Timestamp_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Value_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Value_ICM,embryo_Timestamp_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Value_Line,embryo_Timestamp_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Value_MN2Type,embryo_Timestamp_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Value_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Value_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Value_PN,embryo_Timestamp_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Value_Pulsing,embryo_Timestamp_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Value_Strings,embryo_Timestamp_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Value_TE,embryo_Timestamp_TE,embryo_Name_t2,embryo_Time_t2,embryo_Value_t2,embryo_Timestamp_t2,embryo_Name_t3,embryo_Time_t3,embryo_Value_t3,embryo_Timestamp_t3,embryo_Name_t4,embryo_Time_t4,embryo_Value_t4,embryo_Timestamp_t4,embryo_Name_t5,embryo_Time_t5,embryo_Value_t5,embryo_Timestamp_t5,embryo_Name_t6,embryo_Time_t6,embryo_Value_t6,embryo_Timestamp_t6,embryo_Name_t7,embryo_Time_t7,embryo_Value_t7,embryo_Timestamp_t7,embryo_Name_t8,embryo_Time_t8,embryo_Value_t8,embryo_Timestamp_t8,embryo_Name_t9,embryo_Time_t9,embryo_Value_t9,embryo_Timestamp_t9,embryo_Name_tB,embryo_Time_tB,embryo_Value_tB,embryo_Timestamp_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Value_tEB,embryo_Timestamp_tEB,embryo_Name_tHB,embry

In [76]:
df_long.groupby('presc_grupo_medicamento').agg(
    embrioes = ('oocito_id', 'nunique')
)

,embrioes
presc_grupo_medicamento,
DUPHASTON,2
GONAL,2
GONAPEPTYL,2


In [75]:
df_wide

,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,oocito_ResultadoPGD,oocito_ResultadoPGDDetalhes,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od,nome_medico,tipo_medico,flag_is_external_medical,prontuario,patient_PatientID,patient_PatientIDx,patient_FirstName,patient_LastName,patient_DateOfBirth,patient_YearOfBirth,patient_unit_huntington,treatment_TreatmentName,embryo_EmbryoID,embryo_EmbryoDate,embryo_embryo_number,embryo_Position,embryo_InstrumentNumber,embryo_WellNumber,embryo_FertilizationTime,embryo_FertilizationMethod,embryo_EmbryoFate,embryo_Description,embryo_EmbryoDescriptionID,embryo_KIDScore,embryo_KIDDate,embryo_KIDVersion,embryo_KIDUser,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Value_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Value_EVEN2,embryo_Timestamp_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Value_EVEN4,embryo_Timestamp_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Value_EVEN8,embryo_Timestamp_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Value_FRAG2,embryo_Timestamp_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Value_FRAG4,embryo_Timestamp_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Value_FRAG8,embryo_Timestamp_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Value_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Value_ICM,embryo_Timestamp_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Value_Line,embryo_Timestamp_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Value_MN2Type,embryo_Timestamp_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Value_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Value_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Value_PN,embryo_Timestamp_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Value_Pulsing,embryo_Timestamp_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Value_Strings,embryo_Timestamp_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Value_TE,embryo_Timestamp_TE,embryo_Name_t2,embryo_Time_t2,embryo_Value_t2,embryo_Timestamp_t2,embryo_Name_t3,embryo_Time_t3,embryo_Value_t3,embryo_Timestamp_t3,embryo_Name_t4,embryo_Time_t4,embryo_Value_t4,embryo_Timestamp_t4,embryo_Name_t5,embryo_Time_t5,embryo_Value_t5,embryo_Timestamp_t5,embryo_Name_t6,embryo_Time_t6,embryo_Value_t6,embryo_Timestamp_t6,embryo_Name_t7,embryo_Time_t7,embryo_Value_t7,embryo_Timestamp_t7,embryo_Name_t8,embryo_Time_t8,embryo_Value_t8,embryo_Timestamp_t8,embryo_Name_t9,embryo_Time_t9,embryo_Value_t9,embryo_Timestamp_t9,embryo_Name_tB,embryo_Time_tB,embryo_Value_tB,embryo_Timestamp_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Value_tEB,embryo_Timestamp_tEB,embryo_Name_tHB,embry